I am taking a very famous data set from THE MNIST DATABASE (of handwritten digits), available at http://yann.lecun.com/exdb/mnist/. This was because I wanted to do some image classification for my project; this is a grey-scale data set. However the files are non-standard and require their own numpy package to import.

The files were already set up as a training set of 60,000 images with a corresponding file of labels and a 10,000 image set of testing data (again, with a labels file). The handwritten digits run from 0 to 9, so I have 10 categories. I will use binary encoding so that we only need 4 outputs. Dr. Jakob Streipel helped me write the bits2int and int2bits functions.

No need to sort data because it's already in the two sets.

Importing the data from the files:

In [1]:
from numpy import *
import csv
from scipy.optimize import fmin_bfgs,minimize
import idx2numpy
import time

#global parameters
regParam = 0.03 #use in cost function

In [2]:
#LReLU, optional to leave as ReLU by passing esp=0 
def lrelu(x,b=1,esp=0.01):
    y=[]
    for xi in x:
        if xi<0:
            y.append(esp*xi)
        else:
            y.append(b*xi)
    return array(y)
def lrelu_prime(x,b=1,esp=0.01):
    y=[]
    for xi in x:
        if xi<0:
            y.append(esp)
        else:
            y.append(b)
    return array(y)
#x is input vector, W is weight estimation (use random entries between 0 and 1 for first estimate, call N somewhere)
def feed_forward(x,W):
    W0 = array(W[:k*m]).reshape((k,m))
    b0 = W[k*m:k*m+k]
    W1 = array(W[(m+1)*k:(m+1)*k+n*k]).reshape((n,k))
    b1 = W[(m+1)*k+n*k:]
    z1 = dot(W0,x)+b0
    z2 = dot(W1,a1(z1))+b1
    return a2(z2)

In [3]:
#-------------------------------The rest of the functions we should need are in here----------------------------------
#many of which have been hybrid-ed with Dr. COo
#accepts a column vector argument, returns mean and l2 norm of vector
def normalize(col): 
    l2norm = sqrt(sum((col-mean(col))*(col-mean(col)))/len(col))
    return mean(col),l2norm

#accepts no arguments, returns array of mean values for data & array of norm values
def normalize_cols(inMat):
    means=[]
    norms=[]
    for i in range(m):
        mn,nrm=normalize(inMat[:,i])
        means.append(mn)
        norms.append(nrm)
    meanValues=array(means)
    normValues=array(norms)
    return meanValues,normValues #returns mean and norm values of each column as arrays

#accepts a matrix, returns the normalized matrix
def normalizeByTrain(inMat):
    meanVals,normVals = normalize_cols(inMat)
    #print(normVals)
    normedMat=inMat*0.0
    for i in range(len(normVals)):
         if normVals[i] == 0:
            normVals[i] = 1
    for k in range(len(inMat[:,0])): #for row k,
        normedMat[k]=(inMat[k]-meanVals)/normVals #set the normalized row to the initial row - row of mean values
    return normedMat

#accepts input matrices X and Y and vector W
def ensemble_cost(W,X,Y):
    C = 0.0
    for i in range(len(X[0,:])):
        x=X[i,:]
        y=Y[i,:]
        C += 0.5*(dot(feed_forward(x,W)-y,feed_forward(x,W)-y)+regParam*0.5*linalg.norm(W)**2)
    return C

#Dr. Cooper's logistic function & it's derivative below + his other functions b/c I think they'll work more reliably
def S(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    return 1./(1+exp(-X))

def SPrime(X):
    X = X*(abs(X)<10)+10.*(X>=10)-10.*(X<=-10)
    ex = exp(-X)
    return ex/((1.+ex)*(1.+ex))

def relu(x,epsilon=0.0):
    return x*(x>0)+epsilon*x*(x<=0)

def reluPrime(x,epsilon=0.0):
    return (x>0)+epsilon*(x<=0)

def selu(x,lamda=1,alpha=1):
    return lamda*( x*(x>0)+alpha*(exp(x)-1)*(x<=0))

def seluPrime(x,lamda=1,alpha=1):
    return lamda*((x>0)+alpha*exp(x)*(x<=0))

#accepts input vector x, output vector out, and big W vector, returns a (regularized) gradient for one row
def gradient(x,out,W):#<----borrowed
    W0 = array(W[:nW0]).reshape((k,m))
    W1 = array(W[nW0+k:-n]).reshape((n,k))
    grad = W*0.0
    # Get a(z_2)
    z1 = dot(W0,x)+W[nW0:nW0+k]
    z2 = dot(W1,a1(z1))+W[-n:]
    forward = a2(z2)
    err = forward-out
    # gradient for b_1 bias weights
    finalLayerDeriv = a2p(z2)*err
    grad[-n:] = finalLayerDeriv+0.
    # gradient for W_1 weights
    grad[(nW0+k):-n] = outer(finalLayerDeriv,a1(z1)).flatten()
    # gradient for b_0 bias weights
    firstLayerDeriv = finalLayerDeriv.dot(W1)*a1p(z1)
    grad[nW0:(nW0+k)] = firstLayerDeriv+0.
    # gradient for W_0 weights
    grad[:nW0] = outer(firstLayerDeriv,x).flatten()
    return grad+regParam*W

#accepts vector W, input matrix X and output matrix Y
def gradSum(W,X,Y):
    grad = zeros(nVar)
    for i in range(len(X[:,0])): #X[:,0] is a slice of all the rows in the first column
        grad += gradient(X[i,:],Y[i,:],W)
    return grad
def int2bits(n,bits=4):
    return [int(b) for b in list(format(n, "0"+str(bits)+"b"))]
def bits2int(lst):
    integer =0
    for i in range(len(lst)):
        integer += lst[i]*2**(len(lst)-i-1)
    return integer

Here is the actual neural network code.

In [4]:
# Reading data
inData = idx2numpy.convert_from_file('train-images.idx')
outData = idx2numpy.convert_from_file('train-labels.idx')
inTestX = idx2numpy.convert_from_file('t10k-images.idx')
inTestY = idx2numpy.convert_from_file('t10k-labels.idx')

In [5]:
tempX = []
length,pic1,pic2 = shape(inData)
start = time.time()
for i in range(length):
    tempX.append(inData[i,:,:].reshape(pic1*pic2))
end = time.time()
print(end-start)
trainX = array(tempX)
shape(trainX)

0.10969710350036621


(60000, 784)

In [6]:
n = 4 #n is the number of outputs desired
tempY = []
start = time.time()
for i in range(len(outData)):
    tempY.append(int2bits(int(outData[i]),n))#encoding trainY as binary instead of leaving it as the integer values in outData
end = time.time()
print(end-start)
trainY = array(tempY)

0.20551013946533203


In [7]:
m = len(trainX[0]) #m is the number of inputs, i.e. number of columns in first row
k = int(2*m/3+n) #k is the number of hidden layers
W = random.rand((m + 1)*k+(k + 1)*n) #our first estimate of our W
print('Number of data items: {}'.format(m))
print('Number of hidden cells: {}'.format(k))

nW0 = m*k
nW1 = k*n
nVar = nW0+nW1+k+n

#normalize the vector
trainX = normalizeByTrain(trainX)

Number of data items: 784
Number of hidden cells: 526


In [8]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='lbfgs', activation='logistic',alpha=regParam, hidden_layer_sizes=(k,), random_state=1)
start = time.time()
clf.fit(trainX,trainY)
end = time.time()
print("Time to train:", end-start)

Time to train: 581.8592481613159


In [9]:
tempX = []
length,pic1,pic2 = shape(inTestX)
for i in range(length):
    tempX.append(inTestX[i,:,:].reshape(pic1*pic2))
testX = array(tempX)
shape(testX)
inTestX = normalizeByTrain(testX)

In [10]:
n = 4 #n is the number of outputs desired
tempY = []
for i in range(len(inTestY)):
    tempY.append(int2bits(int(inTestY[i]),n))#encoding trainY as binary instead of leaving it as the integer values in outData
testY = array(tempY)

In [12]:
pred = clf.predict(trainX)
clf.score(trainX,trainY)

1.0

In [13]:
pred = clf.predict(testX)
clf.score(testX,testY)

0.8276

In [24]:
for i in range(30):
    print("Predic:",pred[i,:])
    print("Actual:",testY[i,:])
    print("Number:",inTestY[i])

Predic: [0 1 1 1]
Actual: [0 1 1 1]
Number: 7
Predic: [0 0 1 0]
Actual: [0 0 1 0]
Number: 2
Predic: [0 0 0 1]
Actual: [0 0 0 1]
Number: 1
Predic: [0 0 0 0]
Actual: [0 0 0 0]
Number: 0
Predic: [0 1 0 0]
Actual: [0 1 0 0]
Number: 4
Predic: [0 0 0 1]
Actual: [0 0 0 1]
Number: 1
Predic: [0 0 0 0]
Actual: [0 1 0 0]
Number: 4
Predic: [0 0 1 1]
Actual: [1 0 0 1]
Number: 9
Predic: [1 1 0 0]
Actual: [0 1 0 1]
Number: 5
Predic: [1 0 0 1]
Actual: [1 0 0 1]
Number: 9
Predic: [0 0 0 0]
Actual: [0 0 0 0]
Number: 0
Predic: [0 1 1 0]
Actual: [0 1 1 0]
Number: 6
Predic: [1 0 0 1]
Actual: [1 0 0 1]
Number: 9
Predic: [0 0 0 0]
Actual: [0 0 0 0]
Number: 0
Predic: [0 0 0 1]
Actual: [0 0 0 1]
Number: 1
Predic: [0 1 0 1]
Actual: [0 1 0 1]
Number: 5
Predic: [1 0 0 1]
Actual: [1 0 0 1]
Number: 9
Predic: [0 1 1 1]
Actual: [0 1 1 1]
Number: 7
Predic: [1 0 1 0]
Actual: [0 0 1 1]
Number: 3
Predic: [0 1 0 0]
Actual: [0 1 0 0]
Number: 4
Predic: [1 0 0 1]
Actual: [1 0 0 1]
Number: 9
Predic: [0 1 1 0]
Actual: [0 1 1 0